In [1]:
import os
import matplotlib.pyplot as plt
import shutil
import pandas as pd
import numpy as np

from nnunetv2.paths import nnUNet_results, nnUNet_raw
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
import SimpleITK as sitk

In [2]:
import torch

cuda_version = torch.version.cuda
print("CUDA Version:", cuda_version)


CUDA Version: 11.8


In [3]:
def _flatten(t):
    return [item for sublist in t for item in sublist]


def crop(image, margin, interpolator=sitk.sitkLinear):
    """
    Crops a sitk.Image while retaining correct spacing. Negative margins will lead to zero padding

    Args:
        image:  a sitk.Image
        margin: margins to crop. Single integer or float (percentage crop),
                lists of int/float or nestes lists are supported.
    """
    if isinstance(margin, (list, tuple)):
        assert len(margin) == 3, "expected margin to be of length 3"
    else:
        assert isinstance(margin, (int, float)), "expected margin to be a float value"
        margin = [margin, margin, margin]

    margin = [m if isinstance(m, (tuple, list)) else [m, m] for m in margin]
    old_size = image.GetSize()

    # calculate new origin and new image size
    if all([isinstance(m, float) for m in _flatten(margin)]):
        assert all([m >= 0 and m < 0.5 for m in _flatten(margin)]), "margins must be between 0 and 0.5"
        to_crop = [[int(sz * _m) for _m in m] for sz, m in zip(old_size, margin)]
    elif all([isinstance(m, int) for m in _flatten(margin)]):
        to_crop = margin
    else:
        raise ValueError("Wrong format of margins.")

    new_size = [sz - sum(c) for sz, c in zip(old_size, to_crop)]

    # origin has Index (0,0,0)
    # new origin has Index (to_crop[0][0], to_crop[2][0], to_crop[2][0])
    new_origin = image.TransformIndexToPhysicalPoint([c[0] for c in to_crop])

    # create reference plane to resample image
    ref_image = sitk.Image(new_size, image.GetPixelIDValue())
    ref_image.SetSpacing(image.GetSpacing())
    ref_image.SetOrigin(new_origin)
    ref_image.SetDirection(image.GetDirection())

    return sitk.Resample(image, ref_image, interpolator=interpolator)


# Read and Copy Data

In [36]:
df = pd.read_csv("/home/jaalzate/Tartaglia/Prostate_Tartaglia/New_Dataset/validation_dataset_complete.csv")
path = "/mnt/ceib/datalake/FISABIO_datalake/p0042021"
#groups = df.groupby('subject')


In [37]:
df_t2_axial = df[(df['sequence'] == 'T2w') & (df['orientation']=='ax')]

In [38]:
df_t2_axial

,subject,session,modality,image,orientation,sequence,shape,ProtocolName,SeriesDescription,usable,...,VP,PIR,csPC,F_nacimiento,F_RM,F_TACTO_RECTAL,F_PSA,F_csPC,label_MIDS_session,label_MIDS_subject
1,sub-000063,ses-000063,anat,sub-000063_ses-000063_acq-fse_run-6_bp-p_vp-ax...,ax,T2w,"torch.Size([512, 512, 28])",ax T2,ax T2,True,...,105.698736,3.0,0.0,1947-04-05,2016-07-07,NaN,NaN,14.0,ses-000063,sub-000063
3,sub-000063,ses-000063,anat,sub-000063_ses-000063_acq-fse_run-8_bp-p_vp-ax...,ax,T2w,"torch.Size([512, 512, 28])",ax T2,ax T2,True,...,105.698736,3.0,0.0,1947-04-05,2016-07-07,NaN,NaN,14.0,ses-000063,sub-000063
11,sub-000066,ses-000066,anat,sub-000066_ses-000066_acq-fse_run-8_bp-p_vp-ax...,ax,T2w,"torch.Size([512, 512, 34])",ax T2,ax T2,True,...,NaN,NaN,0.0,1945-03-09,2017-01-27,NaN,-2.0,69.0,ses-000066,sub-000066
22,sub-000184,ses-000184,anat,sub-000184_ses-000184_acq-propeller_run-6_bp-a...,ax,T2w,"torch.Size([512, 512, 55])",Ax T2 PROP. PELVIS,Ax T2 PROP. PELVIS,True,...,46.823400,5.0,1.0,1946-03-12,2018-02-01,NaN,-443.0,162.0,ses-000184,sub-000184
23,sub-000184,ses-000184,anat,sub-000184_ses-000184_acq-propeller_run-3_bp-a...,ax,T2w,"torch.Size([512, 512, 33])",Ax T2 PROPELLER,Ax T2 PROPELLER,True,...,46.823400,5.0,1.0,1946-03-12,2018-02-01,NaN,-443.0,162.0,ses-000184,sub-000184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,sub-000092,ses-000092,anat,sub-000092_ses-000092_acq-fse_run-8_bp-p_vp-ax...,ax,T2w,"torch.Size([512, 512, 28])",ax T2,ax T2,True,...,0.520000,NaN,0.0,1945-10-31,2014-10-02,NaN,-163.0,-90.0,ses-000092,sub-000092
1342,sub-000092,ses-000092,anat,sub-000092_ses-000092_acq-fse_run-13_bp-p_vp-a...,ax,T2w,"torch.Size([512, 512, 28])",ax T2,ax T2,True,...,0.520000,NaN,0.0,1945-10-31,2014-10-02,NaN,-163.0,-90.0,ses-000092,sub-000092
1350,sub-000003,ses-000003,anat,sub-000003_ses-000003_acq-fse_run-10_bp-a_vp-a...,ax,T2w,"torch.Size([512, 512, 26])",ax T2,ax T2,True,...,NaN,NaN,1.0,1945-01-02,2014-03-07,NaN,-42.0,-18.0,ses-000003,sub-000003
1354,sub-000111,ses-000111,anat,sub-000111_ses-000111_acq-fse_run-8_bp-a_vp-ax...,ax,T2w,"torch.Size([512, 512, 28])",ax T2,ax T2,True,...,58.000000,4.0,1.0,1953-01-05,2015-06-24,NaN,NaN,316.0,ses-000111,sub-000111


In [39]:
images_paths = df_t2_axial.copy().apply(lambda x: os.path.join(path, x['subject'], x['session'],'mim-mr','anat', x['image']), axis=1).values

In [40]:
images_paths

array(['/mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000063/ses-000063/mim-mr/anat/sub-000063_ses-000063_acq-fse_run-6_bp-p_vp-ax_T2w.nii.gz',
       '/mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000063/ses-000063/mim-mr/anat/sub-000063_ses-000063_acq-fse_run-8_bp-p_vp-ax_T2w.nii.gz',
       '/mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000066/ses-000066/mim-mr/anat/sub-000066_ses-000066_acq-fse_run-8_bp-p_vp-ax_T2w.nii.gz',
       '/mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000184/ses-000184/mim-mr/anat/sub-000184_ses-000184_acq-propeller_run-6_bp-a_vp-ax_T2w.nii.gz',
       '/mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000184/ses-000184/mim-mr/anat/sub-000184_ses-000184_acq-propeller_run-3_bp-a_vp-ax_T2w.nii.gz',
       '/mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000184/ses-000184/mim-mr/anat/sub-000184_ses-000184_acq-propeller_run-300_bp-a_vp-ax_T2w.nii.gz',
       '/mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000179/ses-000179/mim-mr/anat/sub-000179_

In [41]:
# Create temporal folder to save the images in nnUNet format
tmp_folder_imgs = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_images"
tmp_folder_preds = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predictions"

associated_paths = []

if not os.path.exists(tmp_folder_imgs):
    os.makedirs(tmp_folder_imgs)
if not os.path.exists(tmp_folder_preds):
    os.makedirs(tmp_folder_preds)

# Copying img to model predictions folder.
for i, path in enumerate(images_paths):
    try:
        new_path = os.path.join(tmp_folder_imgs, f"Sample{i}_0000.nii.gz")
        image = sitk.ReadImage(path)

        image = crop(image, margin=[0.2,0.2,0.0])
        associated_paths.append({'original_path': path, 'path_nnunet': new_path})
        sitk.WriteImage(image, new_path)

    except Exception as e:
        print(f"Error in {path}")
        print(e)

associated_paths = pd.DataFrame(associated_paths)

Error in /mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000028/ses-000028/mim-mr/anat/sub-000028_ses-000028_acq-blade_run-9_bp-p_vp-ax_T2w.nii.gz
Exception thrown in SimpleITK ImageFileReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:97:
sitk::ERROR: The file "/mnt/ceib/datalake/FISABIO_datalake/p0042021/sub-000028/ses-000028/mim-mr/anat/sub-000028_ses-000028_acq-blade_run-9_bp-p_vp-ax_T2w.nii.gz" does not exist.


In [82]:
# instantiate the nnUNetPredictor
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    perform_everything_on_device=True,
    device=torch.device('cuda', 0),
    verbose=True,
    verbose_preprocessing=True,
    allow_tqdm=True
)

In [83]:
# initializes the network architecture, loads the checkpoint
results_folder = "/nvmescratch/ceib/Prostate/nnUnet/nnUNet_results/Dataset014_ProstateOwn/nnUNetTrainer_100epochs__nnUNetPlans__3d_fullres"
predictor.initialize_from_trained_model_folder(
    results_folder,
    use_folds=(0,1,2,3,4),
    checkpoint_name='checkpoint_best.pth',
)

In [93]:
# Save Segmentations
predictor.predict_from_files([images_list],
                            images_outdir,
                            save_probabilities=False, overwrite=True,
                            num_processes_preprocessing=2, num_processes_segmentation_export=2,
                            folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict
old shape: (36, 308, 308), new_shape: [ 35 257 257], old_spacing: [3.0, 0.3905999958515167, 0.3905999958515167], new_spacing: [3.089737057685852, 0.46875, 0.46875], fn_data: functools.partial(<function resample_data_or_seg_to_shape at 0x7fe3ea11ce50>, is_seg=False, order=3, order_z=0, force_separate_z=None)

Predicting ProstateSample.nii.gz:
perform_everything_on_device: True
Input shape: torch.Size([1, 35, 257, 257])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 8, image size is torch.Size([35, 257, 257]), tile_size [24, 256, 256], tile_step_size 0.5
steps:
[[0, 11], [0, 1], [0, 1]]
move image to device cuda:0
preallocating results arrays on device cuda:0
running prediction


100%|██████████| 8/8 [00:02<00:00,  3.11it/s]


Input shape: torch.Size([1, 35, 257, 257])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 8, image size is torch.Size([35, 257, 257]), tile_size [24, 256, 256], tile_step_size 0.5
steps:
[[0, 11], [0, 1], [0, 1]]
move image to device cuda:0
preallocating results arrays on device cuda:0
running prediction


100%|██████████| 8/8 [00:02<00:00,  3.13it/s]


Input shape: torch.Size([1, 35, 257, 257])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 8, image size is torch.Size([35, 257, 257]), tile_size [24, 256, 256], tile_step_size 0.5
steps:
[[0, 11], [0, 1], [0, 1]]
move image to device cuda:0
preallocating results arrays on device cuda:0
running prediction


...

100%|██████████| 8/8 [00:02<00:00,  3.13it/s]


Input shape: torch.Size([1, 35, 257, 257])
step_size: 0.5
mirror_axes: (0, 1, 2)
n_steps 8, image size is torch.Size([35, 257, 257]), tile_size [24, 256, 256], tile_step_size 0.5
steps:
[[0, 11], [0, 1], [0, 1]]
move image to device cuda:0
preallocating results arrays on device cuda:0
running prediction


100%|██████████| 8/8 [00:02<00:00,  3.11it/s]


Prediction done
sending off prediction to background worker for resampling and export
done with ProstateSample.nii.gz


[None]

In [50]:
# Predict Array
prediction = predictor.predict_from_files([images_list],
                            None,
                            save_probabilities=True, overwrite=True,
                            num_processes_preprocessing=2, num_processes_segmentation_export=2,
                            folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

There are 1 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 1 cases that I would like to predict

Predicting image of shape torch.Size([1, 23, 341, 339]):
perform_everything_on_device: True


100%|██████████| 4/4 [00:01<00:00,  3.33it/s]


sending off prediction to background worker for resampling

Done with image of shape torch.Size([1, 23, 341, 339]):


# Prediction for Bounding Boxes Data

## Organize Dataset

In [248]:
df_bounding_boxes = pd.read_csv("Prostate_Cancer_TFM/Files/Bounding_Boxes/Only_Bounding.csv")
t2_images = df_bounding_boxes[df_bounding_boxes['modality'] == 'T2'].reset_index(drop=True)

In [249]:
t2_images

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality
0,5586,6040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
1,5586,6040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
2,5596,6050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...,AXIAL T2 TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
3,5700,6173,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/SEG_20240113_211858_507...,AXIAL T2 ALTA RES. (de abajo arriba en próstata),/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
4,5665,6131,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/SEG_20240115_203626_074...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
...,...,...,...,...,...,...,...,...,...,...
226,5636,6097,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/SEG_20240113_202733_764...,T2W_TSE_TRA_IQ,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
227,5613,6070,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/SEG_20240113_203814_089...,AX T2 PROPELLER,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
228,5646,6108,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/SEG_20240113_210319_135...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2
229,5399,5819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...,T2W_TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2


In [250]:
# Create temporal folder to save the images in nnUNet format
tmp_folder_imgs = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_images"
tmp_folder_preds = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predictions"

associated_paths = []

if not os.path.exists(tmp_folder_imgs):
    os.makedirs(tmp_folder_imgs)
if not os.path.exists(tmp_folder_preds):
    os.makedirs(tmp_folder_preds)

# Copying img to model predictions folder.
for i, path in enumerate(t2_images['image_nifti']):

    new_path = os.path.join(tmp_folder_imgs, f"Sample{i}_0000.nii.gz")
    image = sitk.ReadImage(path)

    image = crop(image, margin=[0.2,0.2,0.0])
    associated_paths.append({'original_path': path, 'path_nnunet': new_path})
    sitk.WriteImage(image, new_path)

associated_paths = pd.DataFrame(associated_paths)

In [251]:
associated_paths

,original_path,path_nnunet
0,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
1,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
3,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
4,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
...,...,...
226,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
227,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
228,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...
229,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,Prostate_Cancer_TFM/Prostate_Segmentation/samp...


## Make Predictions

In [42]:
from nnunetv2.paths import nnUNet_results, nnUNet_raw
import torch
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor

In [43]:
folder_images = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_images/"
pred_outdir = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predictions"


In [44]:
# instantiate the nnUNetPredictor
predictor = nnUNetPredictor(
    tile_step_size=0.5,
    use_gaussian=True,
    use_mirroring=True,
    perform_everything_on_device=True,
    device=torch.device('cuda', 0),
    verbose=False,
    verbose_preprocessing=False,
    allow_tqdm=True
)

In [45]:
# initializes the network architecture, loads the checkpoint
results_folder = "/nvmescratch/ceib/Prostate/nnUnet/nnUNet_results/Dataset014_ProstateOwn/nnUNetTrainer_100epochs__nnUNetPlans__3d_fullres"
predictor.initialize_from_trained_model_folder(
    results_folder,
    use_folds=(0,1,2,3,4),
    checkpoint_name='checkpoint_best.pth',
)

/clinicfs/userhomes/jaalzate/nnUNet/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "


In [46]:
# Save Segmentations
predictor.predict_from_files(folder_images,
                            pred_outdir,
                            save_probabilities=False, overwrite=False,
                            num_processes_preprocessing=2, num_processes_segmentation_export=2,
                            folder_with_segs_from_prev_stage=None, num_parts=1, part_id=0)

There are 215 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 215 cases that I would like to predict
overwrite was set to False, so I am only working on cases that haven't been predicted yet. That's 215 cases.

Predicting Sample0:
perform_everything_on_device: True


100%|██████████| 2/2 [00:00<00:00,  3.78it/s]


sending off prediction to background worker for resampling and export
done with Sample0

Predicting Sample1:
perform_everything_on_device: True


100%|██████████| 2/2 [00:00<00:00,  3.78it/s]


sending off prediction to background worker for resampling and export
done with Sample1

Predicting Sample10:
perform_everything_on_device: True


...

100%|██████████| 2/2 [00:00<00:00,  3.78it/s]


sending off prediction to background worker for resampling and export
done with Sample98

Predicting Sample99:
perform_everything_on_device: True


100%|██████████| 2/2 [00:00<00:00,  3.78it/s]


sending off prediction to background worker for resampling and export
done with Sample99


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [282]:
# Move predicted masks to the original folder with the name of the original image
directory = '/nvmescratch/ceib/Prostate/Bounding_Boxes/p0052021/Prostate_Segmentation_AI'
pred_outdir = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predictions"

associated_paths['Path_Segmentation'] = [os.path.join(directory, *path.split('/')[-5:]) for path in associated_paths['original_path']]


for i, row in associated_paths.iterrows():

    path_prediction = os.path.join(pred_outdir, f"Sample{i}.nii.gz")

    new_path = row['Path_Segmentation']

    # Extract the directory part of the new path
    new_directory = os.path.dirname(new_path)

    # Create the new directory if it doesn't exist
    if not os.path.exists(new_directory):
        os.makedirs(new_directory)
    
    shutil.copy(path_prediction,new_path)

In [ ]:
# Save in derivatives in MIDS format

In [278]:
# Remove temporal folders
shutil.rmtree(tmp_folder_imgs)
shutil.rmtree(tmp_folder_preds)



In [283]:
# Associate df_bounding_boxes with the predictions

df_merged = pd.merge(df_bounding_boxes, associated_paths, left_on='image_nifti', right_on='original_path', how='left')
df_merged = df_merged.drop(columns=['original_path', 'path_nnunet'])


In [284]:
df_merged

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality,Path_Segmentation
0,5586,6040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
1,5586,6040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
2,5596,6050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...,AXIAL T2 TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...
3,5700,6173,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/SEG_20240113_211959_757...,AXIAL DW 1500 PROSTATA,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
4,5700,6173,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/SEG_20240113_211927_274...,AXIAL DW 1000 PROSTATA,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
...,...,...,...,...,...,...,...,...,...,...,...
530,5646,6108,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/SEG_20240113_210039_451...,AXI PELVIS DWI B50-800,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
531,5646,6108,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/SEG_20240113_205921_383...,ADC (10^-6 mm²/s):Nov 26 2019 16-59-11 CET,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,ADC,NaN
532,5646,6108,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/SEG_20240113_205854_334...,DW_Synthetic: Ax FOCUS B-50-800 Synthetic B-2000,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,DWI,NaN
533,5399,5819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...,T2W_TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,NaN,NaN,T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...


In [285]:
df_merged.to_csv("Prostate_Cancer_TFM/Files/Bounding_Boxes/Only_Bounding.csv", index=False)

# Prediction for Validation Data

In [25]:
import os
import json
import pandas as pd

In [26]:

# Creating  tables
path = "/mnt/ceib/datalake/FISABIO_datalake/p0042021"
derivatives_path = "/mnt/ceib/datalake/FISABIO_datalake/p0042021/derivatives/creating_adc"
t2_list = []
adc_list = []
dwi_list = []

# Creating a list of all the files in the directory starting with sub-*
subjects = [f for f in os.listdir(path) if f.startswith("sub-")]
derivatives_subs = [f for f in os.listdir(derivatives_path) if f.startswith("sub-")]

for sub in subjects:
    derivative_sessions=None
    if sub in derivatives_subs:
        derivative_sessions = [f for f in os.listdir(os.path.join(derivatives_path, sub)) if f.startswith("ses-")]
    sessions = [f for f in os.listdir(os.path.join(path, sub)) if f.startswith("ses-")]
    for ses in sessions:
        # Check if the anat and dwi paths exist
        anat_path = os.path.join(path, sub, ses,'mim-mr','anat')
        dwi_path = os.path.join(path, sub, ses,'mim-mr','dwi')

        if os.path.exists(anat_path):
            images_anat = [f for f in os.listdir(anat_path) if f.endswith(".nii.gz")]
            for img in images_anat:
                if 'T2w' in img and 'chunk' not in img:
                    json_path = os.path.join(path, sub, ses,'mim-mr','anat', img.replace('.nii.gz', '.json'))
                    with open(json_path) as f:
                        data = json.load(f)
                    #Add json data to the dict image
                    img_dict = {'subject': sub, 'session': ses, 'image': img, 'modality': 'T2w'}
                    img_dict.update(data)
                    t2_list.append(img_dict)
        if os.path.exists(dwi_path):
            images_dwi = [f for f in os.listdir(dwi_path) if f.endswith(".nii.gz")]
            for img in images_dwi:
                if 'bvalue' in img and 'chunk' not in img:
                    json_path = os.path.join(path, sub, ses,'mim-mr','dwi', img.replace('.nii.gz', '.json'))
                    with open(json_path) as f:
                        data = json.load(f)
                    #Add json data to the dict image
                    img_dict = {'subject': sub, 'session': ses, 'image': img, 'modality': 'dwi'}
                    img_dict.update(data)
                    dwi_list.append(img_dict)
                elif 'adc' in img and 'chunk' not in img:
                    json_path = os.path.join(path, sub, ses,'mim-mr','dwi', img.replace('.nii.gz', '.json'))
                    with open(json_path) as f:
                        data = json.load(f)
                    #Add json data to the dict image
                    img_dict = {'subject': sub, 'session': ses, 'image': img, 'modality': 'adc'}
                    img_dict.update(data)
                    adc_list.append(img_dict)
    if derivative_sessions:
        for ses in derivative_sessions:
            der_dwi_path = os.path.join(derivatives_path, sub, ses,'mim-mr','dwi')
            images_dwi = [f for f in os.listdir(der_dwi_path) if f.endswith(".nii.gz")]
            for img in images_dwi:
                json_path = os.path.join(derivatives_path, sub, ses,'mim-mr','dwi', img.replace('.nii.gz', '.json'))
                with open(json_path) as f:
                    data = json.load(f)
                #Add json data to the dict image
                img_dict = {'subject': sub, 'session': ses, 'image': img, 'modality': 'adc'}
                img_dict.update(data)
                adc_list.append(img_dict)


t2w_df = pd.DataFrame(t2_list)
dwi_df = pd.DataFrame(dwi_list)
adc_df = pd.DataFrame(adc_list)

In [27]:
t2w_df

,subject,session,image,modality,AccessionNumber,AcquisitionDateTime,AcquisitionMatrixPE,AcquisitionNumber,AcquisitionTime,AngioFlag,...,WindowCenter,WindowWidth,BodyPartExamined,RawImage,PhaseEncodingSteps,PartialFourier,SequenceName,SliceTiming,InversionTime,Contrast/BolusRoute
0,sub-000063,ses-000063,sub-000063_ses-000063_acq-fse_run-3_bp-p_vp-sa...,T2w,000063,2016-07-07T14:52:47.000000,256,1,14:52:47.000000,N,...,985,1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sub-000063,ses-000063,sub-000063_ses-000063_acq-fse_run-6_bp-p_vp-ax...,T2w,000063,2016-07-07T15:02:32.000000,224,1,15:02:32.000000,N,...,888,1776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sub-000063,ses-000063,sub-000063_ses-000063_acq-fse_run-8_bp-p_vp-ax...,T2w,000063,2016-07-07T15:13:34.000000,224,1,15:13:34.000000,N,...,1286,2573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,sub-000066,ses-000066,sub-000066_ses-000066_acq-fse_run-5_bp-p_vp-sa...,T2w,000066,2017-01-27T15:33:36.000000,256,1,15:33:36.000000,N,...,914,1829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sub-000066,ses-000066,sub-000066_ses-000066_acq-fse_run-8_bp-p_vp-ax...,T2w,000066,2017-01-27T15:44:32.000000,224,1,15:44:32.000000,N,...,960,1921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,sub-000003,ses-000003,sub-000003_ses-000003_acq-fse_run-8_bp-a_vp-sa...,T2w,000003,2014-03-07T08:27:39.000000,256,1,08:27:39.000000,N,...,660,1320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
489,sub-000111,ses-000111,sub-000111_ses-000111_acq-fse_run-8_bp-a_vp-ax...,T2w,000111,2015-06-24T19:55:41.000000,224,1,19:55:41.000000,N,...,865,1731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
490,sub-000111,ses-000111,sub-000111_ses-000111_acq-fse_run-5_bp-a_vp-sa...,T2w,000111,2015-06-24T19:43:50.000000,256,1,19:43:50.000000,N,...,1132,2264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,sub-000120,ses-000120,sub-000120_ses-000120_acq-fse_run-6_bp-a_vp-sa...,T2w,000120,2014-09-24T08:12:51.000000,256,1,08:12:51.000000,N,...,710,1421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
images_paths = t2w_df.copy().apply(lambda x: os.path.join(path, x['subject'], x['session'],'mim-mr','anat', x['image']), axis=1)

# Divide into axial, coronal and sagital images
data_list = []
for path in images_paths.values:
    files = {}

    files['subject'] = path.split('/')[-5]
    files['session'] = path.split('/')[-4]
    
    image_name = path.split('/')[-1]
    if 'sag' in image_name:
        files['sagital'] = path
    elif 'cor' in image_name:
        files['coronal'] = path
    elif 'ax' in image_name:
        files['axial'] = path
    data_list.append(files)

t2w_df = pd.DataFrame(data_list)

In [29]:
t2w_df

,subject,session,sagital,axial,coronal
0,sub-000063,ses-000063,/mnt/ceib/datalake/FISABIO_datalake/p0042021/s...,NaN,NaN
1,sub-000063,ses-000063,NaN,/mnt/ceib/datalake/FISABIO_datalake/p0042021/s...,NaN
2,sub-000063,ses-000063,NaN,/mnt/ceib/datalake/FISABIO_datalake/p0042021/s...,NaN
3,sub-000066,ses-000066,/mnt/ceib/datalake/FISABIO_datalake/p0042021/s...,NaN,NaN
4,sub-000066,ses-000066,NaN,/mnt/ceib/datalake/FISABIO_datalake/p0042021/s...,NaN
...,...,...,...,...,...
488,sub-000003,ses-000003,/mnt/ceib/datalake/FISABIO_datalake/p0042021/s...,NaN,NaN
489,sub-000111,ses-000111,NaN,/mnt/ceib/datalake/FISABIO_datalake/p0042021/s...,NaN
490,sub-000111,ses-000111,/mnt/ceib/datalake/FISABIO_datalake/p0042021/s...,NaN,NaN
491,sub-000120,ses-000120,/mnt/ceib/datalake/FISABIO_datalake/p0042021/s...,NaN,NaN


In [30]:
t2w_df_axial = t2w_df.dropna(subset=['axial'])

In [32]:
t2w_df_axial.subject.unique()

array(['sub-000063', 'sub-000066', 'sub-000184', 'sub-000179',
       'sub-000108', 'sub-000176', 'sub-000103', 'sub-000155',
       'sub-000078', 'sub-000015', 'sub-000042', 'sub-000035',
       'sub-000004', 'sub-000081', 'sub-000026', 'sub-000034',
       'sub-000175', 'sub-000071', 'sub-000018', 'sub-000024',
       'sub-000048', 'sub-000084', 'sub-000127', 'sub-000096',
       'sub-000032', 'sub-000146', 'sub-000168', 'sub-000082',
       'sub-000138', 'sub-000130', 'sub-000160', 'sub-000021',
       'sub-000115', 'sub-000023', 'sub-000062', 'sub-000159',
       'sub-000151', 'sub-000123', 'sub-000093', 'sub-000097',
       'sub-000134', 'sub-000090', 'sub-000121', 'sub-000132',
       'sub-000005', 'sub-000016', 'sub-000033', 'sub-000065',
       'sub-000136', 'sub-000044', 'sub-000142', 'sub-000041',
       'sub-000087', 'sub-000150', 'sub-000085', 'sub-000135',
       'sub-000006', 'sub-000112', 'sub-000030', 'sub-000110',
       'sub-000057', 'sub-000043', 'sub-000047', 'sub-0

In [ ]:
# Create temporal folder to save the images in nnUNet format
tmp_folder_imgs = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_images"
tmp_folder_preds = "Prostate_Cancer_TFM/Prostate_Segmentation/sample_images/tmp_predictions"

associated_paths = []

if not os.path.exists(tmp_folder_imgs):
    os.makedirs(tmp_folder_imgs)
if not os.path.exists(tmp_folder_preds):
    os.makedirs(tmp_folder_preds)

# Copying img to model predictions folder.
for i, path in enumerate(t2w_df_axial['axial']):

    new_path = os.path.join(tmp_folder_imgs, f"Sample{i}_0000.nii.gz")
    image = sitk.ReadImage(path)

    image = crop(image, margin=[0.2,0.2,0.0])
    associated_paths.append({'original_path': path, 'path_nnunet': new_path})
    sitk.WriteImage(image, new_path)

associated_paths = pd.DataFrame(associated_paths)

In [24]:
t2w_df_axial['subject'].unique().shape

(148,)